In [ ]:
import os

from matplotlib import pyplot as plt
import numpy as np
from photutils.aperture import EllipticalAperture

In [ ]:
plt.style.use('guide.mplstyle')

In [ ]:
import sys

seed = os.getenv('GUIDE_RANDOM_SEED', None)
if seed is not None:
    seed = int(seed)
    sys.stdout.write('seed is %i\n' % seed)
else:
    sys.stdout.write('seed is None\n')

noise_rng = np.random.default_rng(seed)

In [ ]:
from convenience_functions import show_image
import math

In [ ]:
synthetic_image = np.zeros([1000,1000])

In [ ]:
show_image(synthetic_image, cmap='gray')

In [ ]:
#from colorspacious import cspace_converter

import matplotlib.pyplot as plt
import numpy as np

import matplotlib as mpl

cmaps = {}

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))


def plot_color_gradients(category, cmap_list):
    # Create figure and adjust figure height to number of colormaps
    nrows = len(cmap_list)
    figh = 0.35 + 0.15 + (nrows + (nrows - 1) * 0.1) * 0.22
    fig, axs = plt.subplots(nrows=nrows + 1, figsize=(6.4, figh))
    fig.subplots_adjust(top=1 - 0.35 / figh, bottom=0.15 / figh,
                        left=0.2, right=0.99)
    axs[0].set_title(f'{category} colormaps', fontsize=14)

    for ax, name in zip(axs, cmap_list):
        ax.imshow(gradient, aspect='auto', cmap=mpl.colormaps[name])
        ax.text(-0.01, 0.5, name, va='center', ha='right', fontsize=10,
                transform=ax.transAxes)

    # Turn off *all* ticks & spines, not just the ones with colormaps.
    for ax in axs:
        ax.set_axis_off()

    # Save colormap list for later.
    cmaps[category] = cmap_list


In [ ]:
plot_color_gradients('Perceptually Uniform Sequential',
                     ['gray', 'viridis', 'plasma', 'inferno', 'magma', 'cividis'])

In [ ]:
def read_noise(image, amount, gain=1):
    """
    Generate simulated read noise.
    
    image: numpy array
        Image whose shape the noise array should match.
    amount : float
        Amount of read noise, in electrons.
    gain : float, optional
        Gain of the camera, in units of electrons/ADU.
    """
    shape = image.shape    
    noise = noise_rng.normal(scale=amount/gain, size=shape)
    
    return noise

In [ ]:
plt.figure()
noise_im = synthetic_image + read_noise(synthetic_image,5)
show_image(noise_im, cmap='gray')

In [ ]:
def bias(image, value, realistic=False):
    """
    Generate simulated bias image.
    
    Parameters
    ----------
    
    image: numpy array
        Image whose shape the bias array should match.
    value: float
        Bias level to add.
    realistic : bool, optional
        If ``True``, add some columns with somewhat higher bias value (a not uncommon thing)
    """
    # This is the whole thing: the bias is really suppose to be a constant offset!
    bias_im = np.zeros_like(image) + value
    
    # If we want a more realistic bias we need to do a little more work. 
    if realistic:
        shape = image.shape
        number_of_colums = 5
        
        # We want a random-looking variation in the bias, but unlike the readnoise the bias should 
        # *not* change from image to image, so we make sure to always generate the same "random" numbers.
        rng = np.random.RandomState(seed=8392)  # 20180520
        columns = rng.randint(0, shape[1], size=number_of_colums)
        # This adds a little random-looking noise into the data.
        col_pattern = rng.randint(0, int(0.1 * value), size=shape[0])
        
        # Make the chosen columns a little brighter than the rest...
        for c in columns:
            bias_im[:, c] = value + col_pattern
            
    return bias_im

In [ ]:
bias_im = bias(noise_im, 1100, realistic=True)
show_image(bias_im, cmap='gray', figsize=(10, 10))
plt.title('Bias plus Noise', fontsize='20')

In [ ]:
def dark_current(image, current, exposure_time, gain=1.0, hot_pixels=False):
    """
    Simulate dark current in a CCD, optionally including hot pixels.
    
    Parameters
    ----------
    
    image : numpy array
        Image whose shape the cosmic array should match.
    current : float
        Dark current, in electrons/pixel/second, which is the way manufacturers typically 
        report it.
    exposure_time : float
        Length of the simulated exposure, in seconds.
    gain : float, optional
        Gain of the camera, in units of electrons/ADU.
    strength : float, optional
        Pixel count in the cosmic rays.    
    """
    
    # dark current for every pixel; we'll modify the current for some pixels if 
    # the user wants hot pixels.
    base_current = current * exposure_time / gain
    
    # This random number generation should change on each call.
    dark_im = noise_rng.poisson(base_current, size=image.shape)
        
    if hot_pixels:
        # We'll set 0.01% of the pixels to be hot; that is probably too high but should 
        # ensure they are visible.
        y_max, x_max = dark_im.shape
        
        n_hot = int(0.0001 * x_max * y_max)
        
        # Like with the bias image, we want the hot pixels to always be in the same places
        # (at least for the same image size) but also want them to appear to be randomly
        # distributed. So we set a random number seed to ensure we always get the same thing.
        rng = np.random.RandomState(16201649)
        hot_x = rng.randint(0, x_max, size=n_hot)
        hot_y = rng.randint(0, y_max, size=n_hot)
        
        hot_current = 10000 * current
        
        dark_im[(hot_y, hot_x)] = hot_current * exposure_time / gain
    return dark_im

In [ ]:
dark_exposure = 100
dark_cur = 0.1
dark_im = dark_current(bias_im, dark_cur, dark_exposure, hot_pixels=True)
show_image(dark_im+bias_im+noise_im, cmap='gray')
title_string = 'Plus dark current, {dark_cur} $e^-$/sec/pix\n{dark_exposure} sec exposure'.format(dark_cur=dark_cur, dark_exposure=dark_exposure)
plt.title(title_string, fontsize='20');

In [ ]:
def sky_background(image, sky_counts, gain=1):
    """
    Generate sky background, optionally including a gradient across the image (because
    some times Moons happen).
    
    Parameters
    ----------
    
    image : numpy array
        Image whose shape the cosmic array should match.
    sky_counts : float
        The target value for the number of counts (as opposed to electrons or 
        photons) from the sky.
    gain : float, optional
        Gain of the camera, in units of electrons/ADU.
    """
    sky_im = noise_rng.poisson(sky_counts * gain, size=image.shape) / gain
    
    return sky_im

In [ ]:
sky_level = 20
sky_only = sky_background(synthetic_image, sky_level)
show_image(sky_only, cmap='gray')
plt.title('Sky background only, {} counts input'.format(sky_level), fontsize=20)

In [ ]:
sky_dark_bias_noise_im = dark_im+bias_im+noise_im + sky_only
show_image(sky_dark_bias_noise_im, cmap='gray')
plt.title('Sky, dark, bias and noise\n(Realistic image of clouds)', fontsize=20);

In [ ]:
def stars(image, number, max_counts=10000, gain=1):
    """
    Add some stars to the image.
    """
    from photutils.datasets import make_random_gaussians_table, make_gaussian_sources_image
    # Most of the code below is a direct copy/paste from
    # https://photutils.readthedocs.io/en/stable/_modules/photutils/datasets/make.html#make_100gaussians_image
    
    flux_range = [max_counts/10, max_counts]
    
    y_max, x_max = image.shape
    xmean_range = [0.1 * x_max, 0.9 * x_max]
    ymean_range = [0.1 * y_max, 0.9 * y_max]
    xstddev_range = [4, 4]
    ystddev_range = [4, 4]
    params = dict([('amplitude', flux_range),
                  ('x_mean', xmean_range),
                  ('y_mean', ymean_range),
                  ('x_stddev', xstddev_range),
                  ('y_stddev', ystddev_range),
                  ('theta', [0, 2*np.pi])])

    sources = make_random_gaussians_table(number, params,
                                          seed=12345)
    
    star_im = make_gaussian_sources_image(image.shape, sources)
    
    return star_im

In [ ]:
stars_only = stars(synthetic_image, 50, max_counts=2000)
show_image(stars_only, cmap='gray', percu=99.9)
plt.title('Stars only'.format(stars_only), fontsize=20)

In [ ]:
stars_with_background = sky_dark_bias_noise_im + stars_only

In [ ]:
show_image(stars_with_background, cmap='gray', percu=99.9)
plt.title('Stars with noise, bias, dark, sky'.format(stars_with_background), fontsize=20)

In [ ]:
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

import image_sim as imsim
from convenience_

In [ ]:
plt.style.use('guide.mplstyle')

In [ ]:
image = np.zeros([500, 500])
gain = 1.0
noise_amount = 150

In [ ]:
gain = 1.0
star_exposure = 30.0
dark_exposure = 60.0
dark = 0.1
sky_counts = 20
bias_level = 1100
read_noise_electrons = 1500
max_star_counts = 10000

In [ ]:
flat = imsim.sensitivity_variations(image)

In [ ]:
real_stars      = imsim.stars(image, 10, max_counts=max_star_counts)
realistic_stars = (real_stars +
                   imsim.dark_current(image, dark, star_exposure, gain=gain, hot_pixels=True) +
                   imsim.bias(image, bias_level, realistic=True) +
                   imsim.read_noise(image, read_noise_electrons, gain=gain)
                  )

In [ ]:
def bias_and_dark(image):
    bias_with_noise = (imsim.bias(image, bias_level, realistic=True) +
                       imsim.read_noise(image, read_noise_electrons, gain=gain))
    dark_with_noise = (imsim.bias(image, bias_level, realistic=True) +
                             imsim.dark_current(image, dark, dark_exposure, gain=gain, hot_pixels=True) +
                             imsim.read_noise(image, read_noise_electrons, gain=gain))
    return (bias_with_noise, dark_with_noise)

In [ ]:
percu=99.9
fig, axes = plt.subplots(3, 2, figsize=(60,50))

show_image(real_stars, cmap='gray', percu=percu, fig=fig, ax=axes[0,0])
calibrated_stars = realistic_stars
show_image(calibrated_stars, cmap='gray', percu=percu, fig=fig, ax=axes[0,1])

for row in range(1,3):
    for col in range(2):
        ax = axes[row,col]
        (f_bias,f_dark) = bias_and_dark(image=image)
        scaled_dark_current = star_exposure * (f_dark - f_bias) / dark_exposure
        calibrated_stars = (calibrated_stars - f_bias - scaled_dark_current) / flat

        show_image(calibrated_stars, cmap='gray', percu=percu, fig=fig, ax=ax)